In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pandas as pd
import os

# Starts a Spark session
spark = (
    SparkSession.builder
        .appName("Taxi vs Rideshare Profitability")
        .config("spark.sql.repl.eagerEval.enabled", False)   
        .config("spark.sql.parquet.cacheMetadata", "true")
        .config("spark.sql.session.timeZone", "Etc/UTC")
        .config("spark.sql.shuffle.partitions", "64")        
        .config("spark.driver.memory", "6g")                
        .config("spark.executor.memory", "6g")
        .getOrCreate()
)

# Define months
months = ["2024-01","2024-02","2024-03","2024-04","2024-05","2024-06"]

# Load in data files 
yellow_files = [f"data/yellow/yellow_tripdata_{m}.parquet" for m in months]
fhvhv_files  = [f"data/fhvhv/fhvhv_tripdata_{m}.parquet"   for m in months]

df_yellow = (
    spark.read.parquet(*yellow_files)
         .withColumn("service_type", lit("yellow"))
)
df_fhvhv = (
    spark.read.parquet(*fhvhv_files)
         .withColumn("service_type", lit("hv_fhv"))
)

# Merge
df = df_yellow.unionByName(df_fhvhv, allowMissingColumns=True)

# External tables (you'll wire these in later for fuel/energy)
electricity = spark.read.csv("data/external/electricity.csv", header=True, inferSchema=True)
fuel        = spark.read.csv("data/external/fuel.csv", header=True, inferSchema=True)



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/16 13:10:18 WARN Utils: Your hostname, LAPTOP-E04ANIN1, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/16 13:10:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/16 13:10:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:

# Preprocess data
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_timestamp, coalesce, unix_timestamp, when, lit, date_format, hour, dayofweek, broadcast
from pyspark.sql.types import IntegerType, DoubleType
from pyspark import StorageLevel
from pyspark.sql import Window

# Reduce memory usage
_needed = [
    "service_type",
    "tpep_pickup_datetime","tpep_dropoff_datetime",
    "pickup_datetime","dropoff_datetime",
    "trip_distance","trip_miles","trip_time",
    "PULocationID","DOLocationID",
    "passenger_count","payment_type",
    "fare_amount","extra","tip_amount",
    "driver_pay","tips"
]
df = df.select([c for c in _needed if c in df.columns])

# Standardise timestamps
df = (
    df.withColumn("pickup_ts",  to_timestamp(coalesce(col("tpep_pickup_datetime"),  col("pickup_datetime"))))
      .withColumn("dropoff_ts", to_timestamp(coalesce(col("tpep_dropoff_datetime"), col("dropoff_datetime"))))
)

# Remove rows with null pickup or dropoff timestamps
df = df.filter(col("pickup_ts").isNotNull() & col("dropoff_ts").isNotNull())

# Standardise location IDs
df = df.withColumn("month", date_format(col("pickup_ts"), "yyyy-MM"))

# Standardise distance and keep positive distances only and not null
df = (
    df.withColumn("distance_mi", coalesce(col("trip_distance"), col("trip_miles")).cast(DoubleType()))
      .filter(col("distance_mi").isNotNull() & (col("distance_mi") > 0))
)

# Standardise trip time and not null
df = df.withColumn(
    "trip_time_s",
    when(col("trip_time").isNotNull(), col("trip_time").cast("double"))
    .otherwise((unix_timestamp(col("dropoff_ts")) - unix_timestamp(col("pickup_ts"))).cast("double"))
)
df = df.filter(col("trip_time_s").isNotNull() & (col("trip_time_s") > 0))

# Deduplicate rows 
dedupe_key = [c for c in ["service_type","pickup_ts","dropoff_ts","PULocationID","DOLocationID","distance_mi","trip_time_s"] if c in df.columns]

# If dedupe_key is empty, we won't deduplicate
if dedupe_key:
    w = Window.partitionBy(["month"] + dedupe_key).orderBy(F.lit(1))
    df = df.withColumn("__rn", F.row_number().over(w)).filter(col("__rn") == 1).drop("__rn")

# Fixed Parameters
CREDIT_CARD_FEE = 0.025 
MAINTENANCE_COST_PER_MILE = 0.15
MAINTENANCE_COST_PER_MILE_HV = 0.15

# Payment type exists
if "payment_type" not in df.columns:
    df = df.withColumn("payment_type", lit(None).cast(IntegerType()))

# Time features 
df = (df
    .withColumn("pickup_hour", hour(col("pickup_ts")))
    .withColumn("pickup_dow", dayofweek(col("pickup_ts")))  
    .withColumn("is_weekend", (col("pickup_dow").isin([1,7])).cast("boolean"))
)

# Add revenue 
rev_yellow = coalesce(col("fare_amount"), lit(0.0)) + coalesce(col("extra"), lit(0.0)) + coalesce(col("tip_amount"), lit(0.0))
rev_hv     = coalesce(col("driver_pay"), lit(0.0)) + coalesce(col("tips"), lit(0.0))
df = df.withColumn(
    "revenue",
    when(col("service_type") == "yellow", rev_yellow).otherwise(rev_hv).cast(DoubleType())
)

# Add costs
maint_rate = when(col("service_type") == "yellow",
                  lit(MAINTENANCE_COST_PER_MILE)
              ).otherwise(
                  lit(MAINTENANCE_COST_PER_MILE_HV)
              )
df = df.withColumn("expense_maintenance", (col("distance_mi") * maint_rate).cast(DoubleType()))

# Credit card fee 
df = df.withColumn(
     "expense_cc_processing",
    when((col("service_type") == "yellow") & (col("payment_type") == 1),
         (lit(CREDIT_CARD_FEE) * col("revenue")).cast(DoubleType()))
    .otherwise(lit(0.0))
)

# Expenses pre-fuel 
df = df.withColumn(
    "expenses_nonfuel",
    (col("expense_maintenance") + col("expense_cc_processing")).cast(DoubleType())
)

# Keep distance
df = df.filter(col("distance_mi") >= 0.1)

# Make sure within month range
df = df.filter( (col("month") >= "2024-01") & (col("month") <= "2024-06") )

# Keep duration >= 60 seconds
df = df.filter(col("trip_time_s") >= 60)

# Keep positive passenger count 
if "passenger_count" in df.columns:
    df = df.filter(
        when(col("service_type") == "yellow", col("passenger_count") > 0)
        .otherwise(True)
    )

# Valid TLC zone IDs 
for c in ["PULocationID", "DOLocationID"]:
    if c in df.columns:
        df = df.filter((col(c) >= 1) & (col(c) <= 263))

# Non-negative money fields 
money_ok = (
    (coalesce(col("fare_amount"), lit(0.0))  >= 0) &
    (coalesce(col("extra"),       lit(0.0))  >= 0) &
    (coalesce(col("tip_amount"),  lit(0.0))  >= 0) &
    (coalesce(col("driver_pay"),  lit(0.0))  >= 0) &
    (coalesce(col("tips"),        lit(0.0))  >= 0)
)
df = df.filter(money_ok)

# Minimum initial fare for Yellow 
df = df.filter(
    when(col("service_type") == "yellow", coalesce(col("fare_amount"), lit(0.0)) >= 1.50)
    .otherwise(True)
)

# Pre-fuel profitability
df = (df
    .withColumn("active_hours", (col("trip_time_s") / 3600.0).cast(DoubleType()))
    .withColumn("net_before_fuel", (col("revenue") - col("expenses_nonfuel")).cast(DoubleType()))
    .withColumn("net_per_hr_before_fuel", (col("net_before_fuel") / col("active_hours")).cast(DoubleType()))
    .withColumn("mph", (col("distance_mi") / (col("trip_time_s")/3600.0)).cast(DoubleType()))
)

# Single-pass outlier trim using 99.9% quantile
stacked = (
    df.select("service_type", F.lit("distance_mi").alias("metric"), col("distance_mi").alias("value"))
      .unionByName(df.select("service_type", F.lit("trip_time_s").alias("metric"), col("trip_time_s").alias("value")))
      .unionByName(df.select("service_type", F.lit("revenue").alias("metric"),     col("revenue").alias("value")))
)
bounds = (
    stacked.groupBy("service_type", "metric")
           .agg(F.expr("percentile_approx(value, 0.999, 10000)").alias("p999"))
)
df = (
    df.alias("t")
      .join(bounds.alias("b1").filter(col("b1.metric") == "distance_mi")
                 .select(col("service_type").alias("s1"), col("p999").alias("p_d")),
            on=[col("t.service_type") == col("s1")], how="left")
      .join(bounds.alias("b2").filter(col("b2.metric") == "trip_time_s")
                 .select(col("service_type").alias("s2"), col("p999").alias("p_t")),
            on=[col("t.service_type") == col("s2")], how="left")
      .join(bounds.alias("b3").filter(col("b3.metric") == "revenue")
                 .select(col("service_type").alias("s3"), col("p999").alias("p_r")),
            on=[col("t.service_type") == col("s3")], how="left")
      .filter( (col("distance_mi") <= F.coalesce(col("p_d"), lit(float("inf")))) &
               (col("trip_time_s") <= F.coalesce(col("p_t"), lit(float("inf")))) &
               (col("revenue")     <= F.coalesce(col("p_r"), lit(float("inf")))) )
      .drop("s1","s2","s3","p_d","p_t","p_r")
)

df = df.repartition(64, "service_type", "month")

# Fuel and energy costs
fuel = fuel.select("month", "price_per_gallon").dropna()
electricity = electricity.select("month", "price_usd_per_kwh").dropna()

# Join fuel and electricity prices
df = (df.join(broadcast(fuel), on="month", how="left")
        .join(broadcast(electricity), on="month", how="left")
)

# Energy assumptions according to EPA and AFDC 
MPG_FHV  = 27.0  
MPG_TAXI = 16.0  

KWH_YELLOW = 0.30
KWH_FHV    = 0.30

YELLOW_EV_PERCENT = 0.00  # Assuming no EVs in Yellow Taxi fleet 
FHV_EV_PERCENT    = 0.10  # Example share for HVFHV

# Per-service parameters as columns 
df = (df
    .withColumn(
        "mpg",
        when(col("service_type") == "yellow", lit(MPG_TAXI))
        .otherwise(lit(MPG_FHV)).cast(DoubleType())
    )
    .withColumn(
        "kwh_per_mile",
        when(col("service_type") == "yellow", lit(KWH_YELLOW))
        .otherwise(lit(KWH_FHV)).cast(DoubleType())
    )
    .withColumn(
        "ev_share",
        when(col("service_type") == "yellow", lit(YELLOW_EV_PERCENT))
        .otherwise(lit(FHV_EV_PERCENT)).cast(DoubleType())
    )
)

# Cost per mile (blend gas vs EV by ev_share)
gas_cpm = (col("price_per_gallon") / col("mpg")).cast(DoubleType())
ev_cpm  = (col("price_usd_per_kwh") * col("kwh_per_mile")).cast(DoubleType())

df = (df
    .withColumn(
        "energy_cost_per_mile",
        ((lit(1.0) - coalesce(col("ev_share"), lit(0.0))) * coalesce(gas_cpm, lit(0.0))) +
        (coalesce(col("ev_share"), lit(0.0)) * coalesce(ev_cpm, lit(0.0)))
    )
    .withColumn("expense_fuel", (col("distance_mi") * col("energy_cost_per_mile")).cast(DoubleType()))
    .withColumn("net_after_fuel", (col("revenue") - col("expenses_nonfuel") - col("expense_fuel")).cast(DoubleType()))
    .withColumn("net_per_hr_after_fuel", (col("net_after_fuel") / col("active_hours")).cast(DoubleType()))
)



+------------+-------+-----------+-----------+-------+-------------------+--------------------+------------------+---------------------+
|service_type|month  |distance_mi|trip_time_s|revenue|expenses_nonfuel   |expense_fuel        |net_after_fuel    |net_per_hr_after_fuel|
+------------+-------+-----------+-----------+-------+-------------------+--------------------+------------------+---------------------+
|yellow      |2024-05|10.7       |1591.0     |55.5   |2.9924999999999997 |2.4101749999999997  |50.097325         |113.35661219358894   |
|yellow      |2024-05|4.1        |1364.0     |32.9   |1.4375             |0.9235249999999999  |30.538974999999997|80.60140029325513    |
|yellow      |2024-05|1.8        |640.0      |16.9   |0.6925             |0.40545000000000003 |15.80205          |88.88653124999999    |
|yellow      |2024-05|2.44       |1660.0     |29.6   |1.106              |0.54961             |27.94439          |60.60229156626505    |
|yellow      |2024-05|1.42       |1537.0 

+------------+-------+-----------+-----------+--------+------------------------+
|service_type|n_trips|avg_revenue|avg_nonfuel|avg_fuel|time_weighted_net_per_hr|
+------------+-------+-----------+-----------+--------+------------------------+
|      yellow| 342721|      23.69|        1.0|    0.68|                   82.65|
+------------+-------+-----------+-----------+--------+------------------------+



+------------+-------+-------------+
|service_type|month  |net_per_hr_tw|
+------------+-------+-------------+
|yellow      |2023-12|147.18       |
|yellow      |2024-01|86.31        |
|yellow      |2024-02|84.31        |
|yellow      |2024-03|83.09        |
|yellow      |2024-04|82.63        |
|yellow      |2024-05|79.8         |
|yellow      |2024-06|81.01        |
+------------+-------+-------------+

